<a href="https://colab.research.google.com/github/msommerfe/CAS-Applied-Data-Science/blob/master/Module6_Deep_Learning/Object_DetectionV8_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import ssl
import urllib
import os
import time

In [3]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.3/716.3 kB 4.3 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO

In [5]:
!nvidia-smi

Sun Mar  3 09:21:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
#!python train.py --img 1280 --batch 16 --epochs 500 --data /content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/YoloTrainingPath.yaml --weights yolov5s.pt
yamlPath = '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/YoloTrainingPath.yaml'
model = YOLO('yolov8n.pt')

results = model.train(data = yamlPath, epochs = 80, imgsz=320)

100%|██████████| 6.23M/6.23M [00:00<00:00, 85.6MB/s]


Ultralytics YOLOv8.1.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/YoloTrainingPath.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

100%|██████████| 755k/755k [00:00<00:00, 16.7MB/s]


Overriding model.yaml nc=80 with nc=18

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/labels.cache... 677 images, 0 backgrounds, 0 corrupt: 100%|██████████| 677/677 [00:00<?, ?it/s]


...

      80/80     0.885G     0.9549     0.6869     0.8804          5        320: 100%|██████████| 43/43 [00:16<00:00,  2.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:13<00:00,  1.63it/s]

                   all        677        735      0.957      0.842      0.914      0.668



80 epochs completed in 0.712 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3009158 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:13<00:00,  1.58it/s]


                   all        677        735      0.955      0.834      0.913      0.675
                 EVN_H        677        343       0.95      0.616      0.784      0.551
                 EVN_V        677        386      0.963      0.885       0.96      0.704
                EVN_HV        677          6      0.951          1      0.995      0.769
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train


In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!cp -av "/content/runs" "/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900"

'/content/runs/detect/train/weights/last.pt' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/weights/last.pt'
'/content/runs/detect/train/weights/best.pt' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/weights/best.pt'
'/content/runs/detect/train/args.yaml' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/args.yaml'
'/content/runs/detect/train/events.out.tfevents.1709457712.8655d8aec19e.997.0' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/events.out.tfevents.1709457712.8655d8aec19e.997.0'
'/content/runs/detect/train/labels_correlogram.jpg' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/labels_correlogram.jpg'
'/content/runs/detect/train/labels.jpg' -> '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/runs/detect/train/labels.jpg'
'/content/runs/detect/train/train_batch0.jpg' -> '/content/drive/MyDri

# Neuer Abschnitt